<a href="https://colab.research.google.com/github/dvschultz/stylegan3/blob/main/SG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN3

By [Derrick Schultz](https://twitter.com/dvsch), with contributions from [crimeacs](https://twitter.com/EarthML1)

Just starting this...expect more updates soon.

## Setup

In [ ]:
!nvidia-smi -L

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Update to python3.8 (modified from [crimeacs](https://twitter.com/EarthML1), thank you!)

In [ ]:
from IPython.display import clear_output

!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y --prefix /usr/local jupyter
!python -m ipykernel install --name "py38" --user
!pip install click -q
!pip install numpy -q
!pip install pillow -q
!pip install torch -q
!pip install scipy -q
!pip install Ninja -q
!pip install imageio -q
!pip install psutil -q
!pip install imageio-ffmpeg -q
clear_output()


In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg3"):
    %cd "/content/drive/MyDrive/colab-sg3/stylegan3/"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg3
    %cd colab-sg3
    !git clone https://github.com/dvschultz/stylegan3
    %cd stylegan3
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan3
    %cd stylegan3
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

## Convert/Create Dataset
Pass a folder of images (just .pngs? TK) to create a zip file.

In [ ]:
!python dataset_tool.py --source=/content/tmp/drawn-gems-1024 --dest=./datasets/drawn-gems-1024.zip

## Training

Working Notes:
- It appears that you must use an SG3 pre-trained model for transfer learning. I _think_ you also want to match config to the pretrained model (`t` with `t`, `r` with `r`).
- For an `A100` I’ve found you can use a `--batch-gpu=8`. For other GPUs I recommend `--batch-gpu=4`.

In [ ]:
!python train.py --help

In [ ]:
!python train.py --outdir=./results --cfg=stylegan3-r --data=./datasets/drawn-gems-1024.zip \
--gpus=1 --batch=32 --batch-gpu=4 --gamma=10.0 --mirror=1 --kimg=5000 --snap=1 \
--resume=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-1024x1024.pkl --metrics=None